In [ ]:
# Importing the necessary and required libraries
from neo4j import GraphDatabase
import pandas as pd
import psycopg2

# Defining Neo4j connection details
neo4j_uri = "neo4j+s://3a7b9db2.databases.neo4j.io:7687"
neo4j_user = "neo4j"
neo4j_password = "3BSDYWZQw1OE9G9O4x-l93fisAeeW4gW7rq1GxpFIi8"
aura_instanceid = "3a7b9db2"
aura_instancename = "Instance01"

# Defining Postgres connection details
"""
POSTGRES_ADDRESS = '35.237.226.12'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres'
POSTGRES_PASSWORD = 'password'
POSTGRES_DBNAME = 'telecommunications_data'
"""

pg_host = "35.237.226.12"
pg_port = "5432"
pg_database = "telecommunications_data"
pg_user = "postgres"
pg_password = "password"

# Defining Neo4j query to extract data
neo4j_query = """
MATCH (c:Customer)-[sub:SUBSCRIBED]->(s:Subscription)-[r:USES]->(sv:Service)
RETURN c.customer_id AS customer_id, s.subscription_id AS subscription_id, sv.service_id AS service_id,
       sub.start_date AS start_date, sub.end_date AS end_date, sub.price AS price
"""

# Defining function to extract data from Neo4j and return a Pandas DataFrame
def extract_data():
    # Connecting to Neo4j
    driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))
    with driver.session() as session:
        result = session.run(neo4j_query)
        records = result.records()
        df = pd.DataFrame.from_records(records, columns=result.keys())
    driver.close()
    return df

# Defining function to transform data
def transform_data(df):
    # Converting the date fields to datetime objects
    df["start_date"] = pd.to_datetime(df["start_date"])
    df["end_date"] = pd.to_datetime(df["end_date"])
    
    # Removing null values
    df.dropna(inplace=True)
     
    return df

# Defining function to load data into Postgres
def load_data(df):
    # Connecting to Postgres
    conn = psycopg2.connect(host=pg_host, port=pg_port, database=pg_database, user=pg_user, password=pg_password)
    # Creating the table if it doesn't exist
    with conn.cursor() as cursor:
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS telecom_data (
            customer_id INTEGER,
            subscription_id INTEGER,
            service_id INTEGER,
            start_date DATE,
            end_date DATE,
            price FLOAT
        )
        """)
    # Inserting data into table
    with conn.cursor() as cursor:
        for _, row in df.iterrows():
            cursor.execute("""
            INSERT INTO telecom_data (customer_id, subscription_id, service_id, start_date, end_date, price)
            VALUES (%s, %s, %s, %s, %s, %s)
            """, (row["customer_id"], row["subscription_id"], row["service_id"], row["start_date"], row["end_date"], row["price"]))
    conn.commit()
    conn.close()

# Defining main function
def main():
    # Extract data from Neo4j
    df = extract_data()
    
    # Transform data using Pandas
    df = transform_data(df)
    
    # Load data into Postgres
    load_data(df)

# Call main function
if __name__ == "__main__":
    main()